In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Notebook imports 

import math
from PIL import Image
import cv2
import random
import matplotlib.pyplot as plt


In [ ]:
dir_path = r'/kaggle/input/dog-breed-identification'

train_data = []
test_data = []

In [ ]:
train_path = os.path.join(dir_path,'train')

train_image = []
for img in os.listdir(train_path):
    img_path = os.path.join(train_path,img)
    img_jpg = cv2.imread(img_path)
    img_resized = cv2.resize(img_jpg,(128,128))
    train_image.append(img_resized)

labels_df = pd.read_csv(os.path.join(dir_path,'labels.csv'))

for i in range(len(labels_df.index)):
    im = train_image[i]
    lab = labels_df.loc[i]['breed']
    train_data.append([im,lab])

print(len(labels_df))
print(len(train_image))
print(len(train_data))

In [26]:
print(train_data[0][0].shape)

(128, 128, 3)


In [ ]:
# random.shuffle(train_data)

In [ ]:
X_train = []
Y_train = []

for img,label in train_data:
#     plt.imshow(img)
#     print(label)
    X_train.append(np.array(img))
    Y_train.append(np.array(label))
    
X_train = np.array(X_train)
# Y_train = np.array(Y_train)

In [25]:
print(X_train.shape)
print(len(Y_train))

(10222, 128, 128, 3)
10222


In [ ]:
X_train = X_train / 255.0

print(X_train[0])

In [24]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
Y_train = np.array(label_encoder.fit_transform(Y_train))

print(Y_train.shape)

(10222,)


In [23]:
# visualizations

print(labels_df.breed.value_counts())

mx_encd = -1
for encd_label in Y_train:
#     print(encd_label)
    mx_encd = max(encd_label,mx_encd)

print('max label encoding = '+str(mx_encd))

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
                       ... 
golden_retriever         67
brabancon_griffon        67
komondor                 67
eskimo_dog               66
briard                   66
Name: breed, Length: 120, dtype: int64
max label encoding = 119


In [ ]:
# imports for CNN

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import BatchNormalization,ZeroPadding2D
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Flatten,Dense
from keras.models import Model,Sequential


In [ ]:
model = Sequential()

model.add(Conv2D(128,(2,2),activation='relu',name='conv1'))
model.add(MaxPooling2D((2,2),name='pool1'))

model.add(Conv2D(128,(2,2),activation='relu',name='conv2'))
model.add(MaxPooling2D((2,2),name='pool2'))

model.add(Conv2D(256,(2,2),activation='relu',name='conv3'))
model.add(MaxPooling2D((2,2),name='pool3'))

model.add(Conv2D(256,(2,2),activation='relu',name='conv4'))
model.add(MaxPooling2D((2,2),name='pool4'))

model.add(Flatten()) 

model.add(Dense(128,input_shape=X_train.shape[1:],activation='relu'))

model.add(Dense(128,activation='relu'))

model.add(Dense(120,activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [22]:
model.fit(X_train,Y_train,epochs=16,validation_split=0.2)

Epoch 1/16
256/256 [==============================] - 8s 31ms/step - loss: 4.7752 - accuracy: 0.0125 - val_loss: 4.7928 - val_accuracy: 0.0117
Epoch 2/16
256/256 [==============================] - 7s 29ms/step - loss: 4.7752 - accuracy: 0.0116 - val_loss: 4.7922 - val_accuracy: 0.0117
Epoch 3/16
256/256 [==============================] - 8s 29ms/step - loss: 4.7753 - accuracy: 0.0117 - val_loss: 4.7923 - val_accuracy: 0.0117
Epoch 4/16
256/256 [==============================] - 7s 29ms/step - loss: 4.7752 - accuracy: 0.0125 - val_loss: 4.7915 - val_accuracy: 0.0117
Epoch 5/16
256/256 [==============================] - 8s 30ms/step - loss: 4.7750 - accuracy: 0.0125 - val_loss: 4.7928 - val_accuracy: 0.0117
Epoch 6/16
256/256 [==============================] - 7s 29ms/step - loss: 4.7751 - accuracy: 0.0125 - val_loss: 4.7918 - val_accuracy: 0.0117
Epoch 7/16
256/256 [==============================] - 8s 30ms/step - loss: 4.7750 - accuracy: 0.0125 - val_loss: 4.7918 - val_accuracy: 0.0117

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 127, 127, 128)     1664      
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 63, 63, 128)       0         
_________________________________________________________________
conv2 (Conv2D)               (None, 62, 62, 128)       65664     
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 31, 31, 128)       0         
_________________________________________________________________
conv3 (Conv2D)               (None, 30, 30, 256)       131328    
_________________________________________________________________
pool3 (MaxPooling2D)         (None, 15, 15, 256)       0         
_________________________________________________________________
conv4 (Conv2D)               (None, 14, 14, 256)       2